## 1.1 Import the required libraries

In [52]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import AssetTypes

In [29]:
# enter details of your AML workspace
subscription_id = "a12569cb-c541-46c4-b11b-14e35067c664"
resource_group = "ml-verse-rg"
workspace = "DvcFlow"

In [30]:
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

## 2.1 Configure the endpoint

In [36]:
endpoint_name = "my-endpoint"

# Example way to define a random name
import datetime

endpoint_name = "endpt-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name = endpoint_name, 
    description="this is a sample endpoint",
    auth_mode="key"
)

ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpt-04261753717265.eastus2.inference.ml.azure.com/score', 'openapi_uri': 'https://endpt-04261753717265.eastus2.inference.ml.azure.com/swagger.json', 'name': 'endpt-04261753717265', 'description': 'this is a sample endpoint', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/a12569cb-c541-46c4-b11b-14e35067c664/resourcegroups/ml-verse-rg/providers/microsoft.machinelearningservices/workspaces/dvcflow/onlineendpoints/endpt-04261753717265', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/a12569cb-c541-46c4-b11b-14e35067c664/providers/Microsoft.MachineLearningServices/locations/eastus2/mfeOperationsStatus/oe:3565b4d4-e664-44e2-897e-8576764e4fb0:6e13e31d-f1c8-4d38-a76c-bdad400ea7fd?api-version=2022-02-01-preview'}, 'print_as_yaml': False, 'id': '/subscriptions/a12569cb-c541-46c4-b11b-14e35067c664/resourceGroups/ml-verse-rg/provider

## 3.1 Configure the deployment

In [43]:
## Optional

# create a myModel deployment
model_version=1

file_model = Model(
    path="../models/model.pkl",
    type=AssetTypes.CUSTOM_MODEL,
    name="dvc-model",
    description="Model created from local file.",
)

## Register Model
ml_client.models.create_or_update(file_model)

Model({'job_name': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'dvc-model', 'description': 'Model created from local file.', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/a12569cb-c541-46c4-b11b-14e35067c664/resourceGroups/ml-verse-rg/providers/Microsoft.MachineLearningServices/workspaces/DvcFlow/models/dvc-model/versions/1', 'Resource__source_path': '', 'base_path': '/Users/zeeshanahmed/Desktop/Python/mlverse-dvc-flows/azure', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x112e1bad0>, 'serialize': <msrest.serialization.Serializer object at 0x112da5210>, 'version': '1', 'latest_version': None, 'path': 'azureml://subscriptions/a12569cb-c541-46c4-b11b-14e35067c664/resourceGroups/ml-verse-rg/workspaces/DvcFlow/datastores/workspaceblobstore/paths/LocalUpload/c5337bde5f474181080ae24c80600c37/model.pkl', 'datastore': None, 'utc_time_created': None

In [55]:
model = Model(path="../sklearn-dummy/sklearn_regression_model.pkl")
env = Environment(
    conda_file="../sklearn-dummy/conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="../sklearn-dummy", scoring_script="score.py"
    ),
    instance_type="Standard_F4s_v2",
    instance_count=1,
)

ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

Check: endpoint endpt-04261753717265 exists


..............................................................................

ManagedOnlineDeployment({'private_network_connection': None, 'package_model': False, 'provisioning_state': 'Succeeded', 'endpoint_name': 'endpt-04261753717265', 'type': 'Managed', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/a12569cb-c541-46c4-b11b-14e35067c664/providers/Microsoft.MachineLearningServices/locations/eastus2/mfeOperationsStatus/od:3565b4d4-e664-44e2-897e-8576764e4fb0:3cc735ce-9d37-42c0-9e53-9c084ebe2726?api-version=2023-04-01-preview'}, 'print_as_yaml': False, 'id': '/subscriptions/a12569cb-c541-46c4-b11b-14e35067c664/resourceGroups/ml-verse-rg/providers/Microsoft.MachineLearningServices/workspaces/DvcFlow/onlineEndpoints/endpt-04261753717265/deployments/blue', 'Resource__source_path': '', 'base_path': '/Users/zeeshanahmed/Desktop/Python/mlverse-dvc-flows/azure', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x113704050>, 'model': '/subscriptions/a1

In [59]:
# myModel deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpt-04261753717265.eastus2.inference.ml.azure.com/score', 'openapi_uri': 'https://endpt-04261753717265.eastus2.inference.ml.azure.com/swagger.json', 'name': 'endpt-04261753717265', 'description': 'this is a sample endpoint', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/a12569cb-c541-46c4-b11b-14e35067c664/resourcegroups/ml-verse-rg/providers/microsoft.machinelearningservices/workspaces/dvcflow/onlineendpoints/endpt-04261753717265', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/a12569cb-c541-46c4-b11b-14e35067c664/providers/Microsoft.MachineLearningServices/locations/eastus2/mfeOperationsStatus/oe:3565b4d4-e664-44e2-897e-8576764e4fb0:0a847e09-6a3a-4fd3-84d9-91f35c5e59fe?api-version=2022-02-01-preview'}, 'print_as_yaml': False, 'id': '/subscriptions/a12569cb-c541-46c4-b11b-14e35067c664/resourceGroups/ml-verse-rg/provider